In [3]:
from pycocotools.coco import COCO
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from os import listdir
import string
import json
from pickle import load
from keras.preprocessing.text import Tokenizer

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

In [5]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		#print(line)        
		#print(len(tokens))        
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

In [47]:
from numpy import argmax
from pickle import load
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
import json

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length, modelfilename):
	json_dict = {}
	actual, predicted, results_list = list(), list(), list()
	ctr = 0
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		print("Processing item %d" %ctr)
		ctr = ctr + 1
		print("Key: %s yhat %s" % (key, yhat))
		print("Actual: %s" % desc_list[0])
		json_dict[int(key)] = yhat.replace("startseq", '').replace("endseq", '').strip()
		dict_element = {"image_id": int(key), "caption":yhat.replace("startseq", '').replace("endseq", '').strip()}
		results_list.append(dict_element)
		print("Saving to json: %s" % json_dict[int(key)])
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split()) 
	print(modelfilename)
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
	print('BLEU %f' % corpus_bleu(actual, predicted))
	json_filename = modelfilename.replace('.','_')
	json_filename = 'validation_results_' + json_filename + '.json'
	#with open('validation_results.json', 'w') as fp:
	with open(json_filename, 'w') as fp:
		#fp.write(json.dumps(json_dict))
		json.dump(results_list,fp)#.replace("'",'"')
	print('saved to ' + json_filename)       


In [40]:
# load test set
filename = '/home/shared/cs231projrepo/cococapt/data/merge/resizeval2017_imagelist.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('/home/shared/cs231projrepo/cococapt/data/merge/descriptions_val2017.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('/home/shared/cs231projrepo/cococapt/data/merge/features_val.pkl', test)
print('Photos: test=%d' % len(test_features))
# prepare sequences
#X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features)

Dataset: 5000
Descriptions: test=5000
Photos: test=5000


In [41]:
tokenizer = load(open('tokenizer_train.pkl', 'rb'))

In [52]:
#Evaluate
# load the model
#filename = 'model-ep004-loss3.586-val_loss3.843.h5'
#model_file = 'm-ep030-loss2.084-val_loss3.792.h5'
#model_file = 'm-ep004-loss3.038-val_loss3.336.h5'
#model_file = 'm-ep006-loss2.784-val_loss3.342.h5'
#model_file = 'm-ep035-loss2.046-val_loss3.840.h5'
model_file = 'm-ep009-loss2.549-val_loss3.422.h5'

model = load_model(model_file)
max_length = 42
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length, model_file)

Processing item 0
Key: 000000397133 yhat startseq group of people standing around buffet table endseq
Actual: startseq man is in kitchen making pizzas endseq
Saving to json: group of people standing around buffet table
Processing item 1
Key: 000000037777 yhat startseq bathroom with sink sink and mirror endseq
Actual: startseq the dining table near the kitchen has bowl of fruit on it endseq
Saving to json: bathroom with sink sink and mirror
Processing item 2
Key: 000000252219 yhat startseq group of people standing next to each other on the street endseq
Actual: startseq person with shopping cart on city street endseq
Saving to json: group of people standing next to each other on the street
Processing item 3
Key: 000000087038 yhat startseq group of people sitting on the street talking on cell phone endseq
Actual: startseq person on skateboard and bike at skate park endseq
Saving to json: group of people sitting on the street talking on cell phone
Processing item 4
Key: 000000174482 yhat 

Processing item 36
Key: 000000219578 yhat startseq dog laying on bed next to dog endseq
Actual: startseq dog and cat lying together on an orange couch endseq
Saving to json: dog laying on bed next to dog
Processing item 37
Key: 000000555705 yhat startseq cat sitting on top of wooden chair endseq
Actual: startseq two cats sitting on top of pair of shoes endseq
Saving to json: cat sitting on top of wooden chair
Processing item 38
Key: 000000443303 yhat startseq cat laying on bed with blanket on it endseq
Actual: startseq cat laying on clothes that are in suitcase endseq
Saving to json: cat laying on bed with blanket on it
Processing item 39
Key: 000000500663 yhat startseq herd of sheep grazing on lush green field endseq
Actual: startseq herd of cattle grazing on lush green field endseq
Saving to json: herd of sheep grazing on lush green field
Processing item 40
Key: 000000418281 yhat startseq herd of cattle grazing on lush green field endseq
Actual: startseq two cows outside one laying d

Processing item 73
Key: 000000486438 yhat startseq sandwich sitting on top of white plate endseq
Actual: startseq plate of donuts with person in the background endseq
Saving to json: sandwich sitting on top of white plate
Processing item 74
Key: 000000400573 yhat startseq man holding doughnut in his hand endseq
Actual: startseq man biting into doughnut with sly look endseq
Saving to json: man holding doughnut in his hand
Processing item 75
Key: 000000109798 yhat startseq plate of food and veggies on plate endseq
Actual: startseq three different donuts one is pink one is brown and one has white sprinkles endseq
Saving to json: plate of food and veggies on plate
Processing item 76
Key: 000000370677 yhat startseq man standing in front of pavilion filled doughnut endseq
Actual: startseq three workers stand next to each other with their baked goods behind them endseq
Saving to json: man standing in front of pavilion filled doughnut
Processing item 77
Key: 000000238866 yhat startseq box of d

Processing item 109
Key: 000000434230 yhat startseq table with lots of wine and bottles of wine endseq
Actual: startseq vendor has lots of different vases for sale endseq
Saving to json: table with lots of wine and bottles of wine
Processing item 110
Key: 000000428454 yhat startseq group of people standing on top of sandy beach endseq
Actual: startseq man skis using sail in open snowpacked field endseq
Saving to json: group of people standing on top of sandy beach
Processing item 111
Key: 000000399462 yhat startseq young boy is standing in the grass with kite endseq
Actual: startseq girl with kite running in the grass endseq
Saving to json: young boy is standing in the grass with kite
Processing item 112
Key: 000000261061 yhat startseq man holding tennis racquet on tennis court endseq
Actual: startseq young lady throwing blue frisbee while standing on lush green field endseq
Saving to json: man holding tennis racquet on tennis court
Processing item 113
Key: 000000168330 yhat startseq s

Processing item 147
Key: 000000029393 yhat startseq brown horse standing next to brown calf endseq
Actual: startseq large brown dog standing on the side of white wood fence endseq
Saving to json: brown horse standing next to brown calf
Processing item 148
Key: 000000278848 yhat startseq group of people walking down street holding umbrellas endseq
Actual: startseq group of people standing outside of bus endseq
Saving to json: group of people walking down street holding umbrellas
Processing item 149
Key: 000000166391 yhat startseq traffic light hanging over street covered in tall trees endseq
Actual: startseq large and wide street covered in lots of traffic lights endseq
Saving to json: traffic light hanging over street covered in tall trees
Processing item 150
Key: 000000048153 yhat startseq woman sitting on beach holding surfboard endseq
Actual: startseq foot wearing pointy pink flat shoe rested upon bench endseq
Saving to json: woman sitting on beach holding surfboard
Processing item 

...

Processing item 4837
Key: 000000080274 yhat startseq an elephant standing next to bigger elephant in the jungle endseq
Actual: startseq two elephants that are standing next to each other endseq
Saving to json: an elephant standing next to bigger elephant in the jungle
Processing item 4838
Key: 000000520832 yhat startseq man riding skateboard down street endseq
Actual: startseq aa man walks down city street next to shop window endseq
Saving to json: man riding skateboard down street
Processing item 4839
Key: 000000250137 yhat startseq group of people standing next to each other endseq
Actual: startseq woman reaching in her purse while holding an umbrella endseq
Saving to json: group of people standing next to each other
Processing item 4840
Key: 000000082821 yhat startseq group of boats floating across body of water endseq
Actual: startseq paddle boats parked at the edge of river or lake endseq
Saving to json: group of boats floating across body of water
Processing item 4841
Key: 000000

Processing item 4873
Key: 000000306437 yhat startseq an old modern modern modern kitchen with some trees endseq
Actual: startseq man riding skateboard up flight of steps endseq
Saving to json: an old modern modern modern kitchen with some trees
Processing item 4874
Key: 000000116206 yhat startseq plate of food and vegetables sitting on table endseq
Actual: startseq wooden cutting board filled with chopped vegetables endseq
Saving to json: plate of food and vegetables sitting on table
Processing item 4875
Key: 000000393838 yhat startseq pizza sitting on top of stove top endseq
Actual: startseq pot full of vegetables and ready to be cooked endseq
Saving to json: pizza sitting on top of stove top
Processing item 4876
Key: 000000229553 yhat startseq young boy riding skateboard down street endseq
Actual: startseq man sitting on top of skateboard on street endseq
Saving to json: young boy riding skateboard down street
Processing item 4877
Key: 000000171382 yhat startseq man riding skateboard

Processing item 4909
Key: 000000246522 yhat startseq large clock tower with clock on it endseq
Actual: startseq very tall church with clock below tower endseq
Saving to json: large clock tower with clock on it
Processing item 4910
Key: 000000446703 yhat startseq cat sitting on the ground next to cat endseq
Actual: startseq teddy bear under some furniture that appears to be turned on its side endseq
Saving to json: cat sitting on the ground next to cat
Processing item 4911
Key: 000000367095 yhat startseq man sitting at table with laptop computer endseq
Actual: startseq man that is sitting in front of laptop endseq
Saving to json: man sitting at table with laptop computer
Processing item 4912
Key: 000000529148 yhat startseq man sitting on desk with laptop computer endseq
Actual: startseq young children sharing laptop in messy room with several laptops books and papers endseq
Saving to json: man sitting on desk with laptop computer
Processing item 4913
Key: 000000480212 yhat startseq pers

Processing item 4943
Key: 000000547336 yhat startseq man sitting on top of wooden bench endseq
Actual: startseq the woman sits on the curb writing on notepad near fire hydrant endseq
Saving to json: man sitting on top of wooden bench
Processing item 4944
Key: 000000226154 yhat startseq bus driving down street next to trees endseq
Actual: startseq green and white bus on street next to dirt area endseq
Saving to json: bus driving down street next to trees
Processing item 4945
Key: 000000394275 yhat startseq bus driving down street next to building endseq
Actual: startseq blue bus driving down street past park endseq
Saving to json: bus driving down street next to building
Processing item 4946
Key: 000000229753 yhat startseq two giraffes in the foreground on sunny day endseq
Actual: startseq two tall giraffes are next to bare trees endseq
Saving to json: two giraffes in the foreground on sunny day
Processing item 4947
Key: 000000261712 yhat startseq giraffe standing next to stone fence en

Processing item 4980
Key: 000000477955 yhat startseq man flying kite on top of sandy beach endseq
Actual: startseq para sailor launching his sail at the beach endseq
Saving to json: man flying kite on top of sandy beach
Processing item 4981
Key: 000000128675 yhat startseq man riding surfboard on top of wave endseq
Actual: startseq man holding one water ski at the lake endseq
Saving to json: man riding surfboard on top of wave
Processing item 4982
Key: 000000402720 yhat startseq group of people sitting around table with food endseq
Actual: startseq trio of men drinking wine at restaurant table endseq
Saving to json: group of people sitting around table with food
Processing item 4983
Key: 000000179141 yhat startseq woman standing next to woman in front of cake endseq
Actual: startseq four people are smiling together over box of pizza endseq
Saving to json: woman standing next to woman in front of cake
Processing item 4984
Key: 000000353096 yhat startseq computer monitor sitting on top of